### This version only optimizes the replacement schedule for a single vehicle

### Todo:
- Get all possible permutations of a replacement schedule (with 1 replacement, 2, 3)
- make replacement schedules flexible with more than 15 year timelines (i.e. more than 3 replacements)
- vehicle age matrix
- vehicle_mileage matrix
- odometer matrix
- acquisition_cost matrix (pct change factor)
- consumables_cost matrix
- maintenance_cost matrix

In [202]:
import numpy as np
from sympy.utilities.iterables import multiset_permutations

np.set_printoptions(edgeitems=15,linewidth=600)
# np.core.arrayprint._line_width = 400

In [203]:
150000/4

37500.0

In [204]:
#todo: will likely need a generalizable method for computing the maximum number of replacements possible
oneReplacement = np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
twoReplacements = np.array([1,1,0,0,0,0,0,0,0,0,0,0,0,0,0])
threeReplacements = np.array([1,1,1,0,0,0,0,0,0,0,0,0,0,0,0])

oneReplacement = list(multiset_permutations(oneReplacement))
twoReplacements = list(multiset_permutations(twoReplacements))
threeReplacements = list(multiset_permutations(threeReplacements))

replacementSchedules = np.array(oneReplacement+twoReplacements+threeReplacements)

In [205]:
len(replacementSchedules)*1440

828000

In [206]:
a = np.array([[1, 2], [1, 2]])
print(a)

[[1 2]
 [1 2]]


In [207]:
a[:, :, np.newaxis]

array([[[1],
        [2]],

       [[1],
        [2]]])

In [208]:
replacementSchedules#[50:80]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       ...,
       [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 0,

In [209]:
#opposite of replacement -- may want to use this at some point
keepSchedules = (replacementSchedules-1)*-1
keepSchedules#[50:80]

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       ...,
       [0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 1,

In [210]:
def initialize_vehicle_age(keepSchedules,startingAge=0):
    """Gets the vehicle age building process started. Produces a matrix for the age of the vehicle according to the replacement schedule, which is later fixed by get_vehicle_age"""
    vehicle_age = startingAge+np.cumsum(keepSchedules,axis=1)*keepSchedules
    vehicle_age[vehicle_age==startingAge] = 0 #fixes the fact that replaced vehicles start at 0 (if this wasn't here they would start at the starting age)
    return vehicle_age


def get_vehicle_age(vehicle_age=None,k=1,startingAge=0):
    if k==1:
        vehicle_age = initialize_vehicle_age(keepSchedules,startingAge)

    diff = np.diff(vehicle_age,axis=1)
    diffMask = np.append(np.ones(shape=(575,1)),diff,axis=1)>1
    vehicle_age[diffMask]=k

    if vehicle_age[diffMask].size==0:
        return vehicle_age
    else:
        return get_vehicle_age(vehicle_age,k=k+1)

vehicle_age = get_vehicle_age(startingAge=0)#[50:80]

In [211]:
vehicle_age

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  0],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  0,  1],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  0,  1,  2],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,  0,  1,  2,  3],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9,  0,  1,  2,  3,  4,  5],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  0,  1,  2,  3,  4,  5,  6],
       [ 1,  2,  3,  4,  5,  6,  7,  0,  1,  2,  3,  4,  5,  6,  7],
       [ 1,  2,  3,  4,  5,  6,  0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 1,  2,  3,  4,  5,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 1,  2,  3,  4,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 1,  2,  3,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
       [ 1,  2,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
       [ 0,  1,  2,  3,  4,  5,  6

In [212]:
#start of something for determining infeasible schedules. will come back to this. 
x = np.transpose((vehicle_age==0).nonzero())
x[:,1] = x[:,1]-1

In [213]:
vehicle_age[1,12]

13

In [214]:
vehicle_age

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  0],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  0,  1],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  0,  1,  2],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,  0,  1,  2,  3],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9,  0,  1,  2,  3,  4,  5],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  0,  1,  2,  3,  4,  5,  6],
       [ 1,  2,  3,  4,  5,  6,  7,  0,  1,  2,  3,  4,  5,  6,  7],
       [ 1,  2,  3,  4,  5,  6,  0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 1,  2,  3,  4,  5,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 1,  2,  3,  4,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 1,  2,  3,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
       [ 1,  2,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
       [ 0,  1,  2,  3,  4,  5,  6

In [215]:
x

array([[  0,  13],
       [  1,  12],
       [  2,  11],
       [  3,  10],
       [  4,   9],
       [  5,   8],
       [  6,   7],
       [  7,   6],
       [  8,   5],
       [  9,   4],
       [ 10,   3],
       [ 11,   2],
       [ 12,   1],
       [ 13,   0],
       [ 14,  -1],
       ...,
       [570,  -1],
       [570,   0],
       [570,   5],
       [571,  -1],
       [571,   0],
       [571,   4],
       [572,  -1],
       [572,   0],
       [572,   3],
       [573,  -1],
       [573,   0],
       [573,   2],
       [574,  -1],
       [574,   0],
       [574,   1]], dtype=int64)

In [216]:
x,y = np.where(vehicle_age==0)#,axis=0)

In [217]:
np.array((x,y)).shape

(2, 1590)

In [218]:
#vehicle_mileage matrix 
# vehicle_mileage = np.repeat(np.round(np.random.normal(loc=10000,scale=2000,size=(1, 15))),575,axis=0)
vehicle_mileage = np.round(np.random.normal(loc=10000,scale=2000)*np.ones(shape=(575,15)))

In [219]:
#matrix showing what the odometer reading will be under each schedule
vehicle_odometer = vehicle_mileage*vehicle_age
vehicle_odometer

array([[ 10876.,  21752.,  32628.,  43504.,  54380.,  65256.,  76132.,  87008.,  97884., 108760., 119636., 130512., 141388., 152264.,      0.],
       [ 10876.,  21752.,  32628.,  43504.,  54380.,  65256.,  76132.,  87008.,  97884., 108760., 119636., 130512., 141388.,      0.,  10876.],
       [ 10876.,  21752.,  32628.,  43504.,  54380.,  65256.,  76132.,  87008.,  97884., 108760., 119636., 130512.,      0.,  10876.,  21752.],
       [ 10876.,  21752.,  32628.,  43504.,  54380.,  65256.,  76132.,  87008.,  97884., 108760., 119636.,      0.,  10876.,  21752.,  32628.],
       [ 10876.,  21752.,  32628.,  43504.,  54380.,  65256.,  76132.,  87008.,  97884., 108760.,      0.,  10876.,  21752.,  32628.,  43504.],
       [ 10876.,  21752.,  32628.,  43504.,  54380.,  65256.,  76132.,  87008.,  97884.,      0.,  10876.,  21752.,  32628.,  43504.,  54380.],
       [ 10876.,  21752.,  32628.,  43504.,  54380.,  65256.,  76132.,  87008.,      0.,  10876.,  21752.,  32628.,  43504.,  54380.,  6

### Find and filter out infeasible schedules
- infeasible in the sense that they have a replacement happening in years where the vehicle is both under 6 years old and doesn't yet have 150k miles on it

In [220]:
odometer_diff = np.diff(vehicle_odometer)
odometer_check = (odometer_diff>-100000) & (odometer_diff<=0)

In [221]:
vehicle_odometer[:10]

array([[ 10876.,  21752.,  32628.,  43504.,  54380.,  65256.,  76132.,  87008.,  97884., 108760., 119636., 130512., 141388., 152264.,      0.],
       [ 10876.,  21752.,  32628.,  43504.,  54380.,  65256.,  76132.,  87008.,  97884., 108760., 119636., 130512., 141388.,      0.,  10876.],
       [ 10876.,  21752.,  32628.,  43504.,  54380.,  65256.,  76132.,  87008.,  97884., 108760., 119636., 130512.,      0.,  10876.,  21752.],
       [ 10876.,  21752.,  32628.,  43504.,  54380.,  65256.,  76132.,  87008.,  97884., 108760., 119636.,      0.,  10876.,  21752.,  32628.],
       [ 10876.,  21752.,  32628.,  43504.,  54380.,  65256.,  76132.,  87008.,  97884., 108760.,      0.,  10876.,  21752.,  32628.,  43504.],
       [ 10876.,  21752.,  32628.,  43504.,  54380.,  65256.,  76132.,  87008.,  97884.,      0.,  10876.,  21752.,  32628.,  43504.,  54380.],
       [ 10876.,  21752.,  32628.,  43504.,  54380.,  65256.,  76132.,  87008.,      0.,  10876.,  21752.,  32628.,  43504.,  54380.,  6

In [222]:
odometer_diff

array([[  10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876., -152264.],
       [  10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876., -141388.,   10876.],
       [  10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876., -130512.,   10876.,   10876.],
       [  10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876., -119636.,   10876.,   10876.,   10876.],
       [  10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876., -108760.,   10876.,   10876.,   10876.,   10876.],
       [  10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,  -97884.,   10876.,   10876.,   10876.,   10876.,   10876.],
       [  10876.,   10876.,   10876.,   10876.,   10876.,   10876.,   10876.,  -87008.,   10876.,   10876.

In [223]:
odometer_check

array([[False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False,  True, False, False, False, False, False],
       [False, False, False, False, False, False, False,  True, False, False, False, False, False, False],
       [False, False, False, False, False, False,  True, False, False, False, False, False, False, False],
       [False, False, False, False, False,  True, False, False, False, False, False, False, False, False],
       [False, False, False, False,  

In [224]:
age_diff = np.diff(vehicle_age)
age_check = (age_diff>-6) & (age_diff<=0)#.any()


In [225]:
vehicle_age[:10]

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  0],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  0,  1],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  0,  1,  2],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,  0,  1,  2,  3],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9,  0,  1,  2,  3,  4,  5],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  0,  1,  2,  3,  4,  5,  6],
       [ 1,  2,  3,  4,  5,  6,  7,  0,  1,  2,  3,  4,  5,  6,  7],
       [ 1,  2,  3,  4,  5,  6,  0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 1,  2,  3,  4,  5,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9]])

In [226]:
age_diff

array([[  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1, -14],
       [  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1, -13,   1],
       [  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1, -12,   1,   1],
       [  1,   1,   1,   1,   1,   1,   1,   1,   1,   1, -11,   1,   1,   1],
       [  1,   1,   1,   1,   1,   1,   1,   1,   1, -10,   1,   1,   1,   1],
       [  1,   1,   1,   1,   1,   1,   1,   1,  -9,   1,   1,   1,   1,   1],
       [  1,   1,   1,   1,   1,   1,   1,  -8,   1,   1,   1,   1,   1,   1],
       [  1,   1,   1,   1,   1,   1,  -7,   1,   1,   1,   1,   1,   1,   1],
       [  1,   1,   1,   1,   1,  -6,   1,   1,   1,   1,   1,   1,   1,   1],
       [  1,   1,   1,   1,  -5,   1,   1,   1,   1,   1,   1,   1,   1,   1],
       [  1,   1,   1,  -4,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
       [  1,   1,  -3,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
       [  1,  -2,   1,   1,   1,   1,   1,   1,   1,

In [227]:
age_check

array([[False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False,  

In [228]:
both_check = odometer_check*age_check

In [229]:
both_check

array([[False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False, False, False, False, False, False],
       [False, False, False, False,  

In [230]:
infeasible_filter = both_check.any(axis=1)

In [231]:
replacementSchedules[~infeasible_filter]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0,

In [232]:
def find_infeasible_schedules():
    
    return None

### Fuel cost

In [233]:
#ICE
#fuel $ = mileage/mpg*cpg
cpg = 2.25
mpg = 22
fuel_cost_ice = vehicle_mileage/mpg*cpg

In [234]:
#EV
#fuel $ = mileage/mpeg * cpeg
cpeg = 1.2
mpeg = 100
fuel_cost_ev = vehicle_mileage/mpeg*cpeg

#!having trouble getting the masking to work here. Need to do some more research on how to get this to work. 

In [235]:
firstReplacements = np.argmax(replacementSchedules==1,axis=1)

In [236]:
firstReplacements.shape

(575,)

In [237]:
firstReplacements

array([14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0, 13, 12, 12, 11, 11, 11, 10, 10, 10, 10,  9,  9,  9,  9,  9,  8,  8,  8,  8,  8,  8,  7,  7,  7,  7,  7,  7,  7,  6,  6,  6,  6,  6,  6,  6,  6,  5,  5,  5,  5,  5,  5,  5,  5,  5,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 12, 11, 11, 11, 10, 10, 10, 10, 10, 10,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  4,  4,  4,  4,  4,  4,  4

In [246]:
end = np.repeat(np.array(14),575)

In [247]:
schedule = np.indices((575,1))

In [249]:
replacementSchedules[schedule,firstReplacements:end] = 1

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
import numpy as np
>>> a = np.zeros((4,5))
>>> vals = 1
>>> pos = [(0,4),(1,2),(2,4),(3,4)]
>>> rows, cols = zip(*pos)
>>> a[2, 2:4] = vals
# >>> a
a

In [ ]:
rows

In [ ]:
replacementSchedules[:,:14]# = 1

In [ ]:
aa.shape

In [ ]:
replacementSchedules

### Old stuff from vehicle age recursion 

In [ ]:
#vehicle_age matrix tracks the age of the vehicle throughout each schedule
# vehicle_age = 

x = 3+np.cumsum(keepSchedules,axis=1)*keepSchedules #3+ would be the age of the vehicle currently
x[x==3] = 0
u = np.diff(x,axis=1)
x[np.append(np.ones(shape=(575,1)),u,axis=1)>1]=1


In [ ]:
x

In [ ]:
u = np.diff(x,axis=1)
x[np.append(np.ones(shape=(575,1)),u,axis=1)>1]=2

In [ ]:
u = np.diff(x,axis=1)
x[np.append(np.ones(shape=(575,1)),u,axis=1)>1]=3

In [ ]:
u = np.diff(x,axis=1)
x[np.append(np.ones(shape=(575,1)),u,axis=1)>1] = 4

In [ ]:
u = np.diff(x,axis=1)
x[np.append(np.ones(shape=(575,1)),u,axis=1)>1] = 5

In [ ]:
u = np.diff(x,axis=1)
x[np.append(np.ones(shape=(575,1)),u,axis=1)>1] = 6

In [ ]:
u = np.diff(x,axis=1)
x[np.append(np.ones(shape=(575,1)),u,axis=1)>1] = 7

In [ ]:
u = np.diff(x,axis=1)
x[np.append(np.ones(shape=(575,1)),u,axis=1)>1] = 8

In [ ]:
u = np.diff(x,axis=1)
x[np.append(np.ones(shape=(575,1)),u,axis=1)>1] = 9

In [ ]:
u = np.diff(x,axis=1)
x[np.append(np.ones(shape=(575,1)),u,axis=1)>1] = 10

In [ ]:
u = np.diff(x,axis=1)
x[np.append(np.ones(shape=(575,1)),u,axis=1)>1] = 11

In [ ]:
u = np.diff(x,axis=1)
x[np.append(np.ones(shape=(575,1)),u,axis=1)>1] = 12

In [ ]:
u = np.diff(x,axis=1)
x[np.append(np.ones(shape=(575,1)),u,axis=1)>1] = 13

In [ ]:
u = np.diff(x,axis=1)
x[np.append(np.ones(shape=(575,1)),u,axis=1)>1] = 14

In [ ]:
u = np.diff(x,axis=1)
x[np.append(np.ones(shape=(575,1)),u,axis=1)>1]

In [ ]:
x

In [ ]:
u = np.diff(x)
x[np.append(np.ones(shape=(1,)),u,axis=0)>1] = 2

In [ ]:
x

In [ ]:
u = np.diff(x)
x[np.append(np.ones(shape=(1,)),u,axis=0)>1] = 3

In [ ]:
x

In [ ]:
u = np.diff(x)
x[np.append(np.ones(shape=(1,)),u,axis=0)>1] = 4

In [ ]:
x

In [ ]:
u = np.diff(x)
x[np.append(np.ones(shape=(1,)),u,axis=0)>1] = 5

In [ ]:
x

In [ ]:
u = np.diff(x)
x[np.append(np.ones(shape=(1,)),u,axis=0)>1] = 6

In [ ]:
x

In [ ]:
#acquisition cost timeline
acquisition_cost = 30000*replacementSchedules
acquisition_cost

In [ ]:
acquisition_cost[45,:]

In [ ]:
allVehiclesReplacementSchedules = np.repeat(replacementSchedules[np.newaxis,:, :], 1440, axis=0)
print(allVehiclesReplacementSchedules.shape)
allVehiclesReplacementSchedules

In [239]:
v,s,y = allVehiclesReplacementSchedules.shape
v*s

NameError: name 'allVehiclesReplacementSchedules' is not defined